# Overview
This blog documents the preparation steps for obtaining the 3 Musketeer passwords and deploying both the frontend and backend applications. The goal is to ensure a smooth deployment process without breaking the machine.

# Prerequisites
### - **Review Documentation**
- AWS Deployment Guide: [BI4](https://nighthawkcoders.github.io/portfolio_2025/2024/12/03/aws-deployment_IPYNB_2_.html)
### - **Prepare Code Repository**
- 